### yes24 - 국내도서 새로나온 상품
- title, author, company, price, image, summary

In [2]:
import requests
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup

In [3]:
base_url = 'https://www.yes24.com'
main_url = '/Product/Category/NewProduct?categoryNumber=001&pageSize=24&newProductType=NEW'
url = f'{base_url}{main_url}&pageNumber=1'
# header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36'}
# result = requests.get(url, headers=header)
result = requests.get(url)
soup = BeautifulSoup(result.text)

In [3]:
url

'https://www.yes24.com/Product/Category/NewProduct?categoryNumber=001&pageSize=24&newProductType=NEW&pageNumber=1'

In [4]:
lis = soup.select('#yesNewList > li')
len(lis)

24

#### 하나의 데이터로 원하는 정보 추출

In [16]:
li = lis[1]

In [17]:
# sub-page
sub_page = li.select_one('.img_grp > a')['href']
sub_url = f'{base_url}{sub_page}'

In [18]:
res = requests.get(sub_url)
book_soup = BeautifulSoup(res.text)

In [19]:
# image
imageUrl = book_soup.select_one('img.gImg')['src']
imageUrl

'https://image.yes24.com/goods/138627760/XL'

In [20]:
# title
title = book_soup.select_one('h2.gd_name').text.strip()
title

'참된 회개에서 죄사함까지'

In [21]:
# author
author = book_soup.select_one('.gd_auth').text.strip()
author

'홍순구 저'

In [22]:
# company
company = book_soup.select_one('.gd_pub').text.strip()
company

'BOOKK(부크크)'

In [23]:
# price
price = book_soup.select_one('em.yes_m').text.strip()
price = int(price[:-1].replace(',', ''))
price

16000

In [24]:
# summary
summary = book_soup.select_one('.txtContentText').get_text(separator='\n').strip()
summary

AttributeError: 'NoneType' object has no attribute 'get_text'

#### 한 페이지에 있는 데이터
- 요약이 없는 데이터는 수집하지 않음

In [5]:
lines = []
for index, li in enumerate(lis):
    print(index, end='  ')
    try:
        sub_page = li.select_one('.img_grp > a')['href']
        sub_url = f'{base_url}{sub_page}'
        res = requests.get(sub_url)
        book_soup = BeautifulSoup(res.text)
        imageUrl = book_soup.select_one('img.gImg')['src']
        title = book_soup.select_one('h2.gd_name').text.strip()
        author = book_soup.select_one('.gd_auth').text.strip()
        company = book_soup.select_one('.gd_pub').text.strip()
        price = book_soup.select_one('em.yes_m').text.strip()
        price = int(price[:-1].replace(',', ''))
        summary = book_soup.select_one('.txtContentText').get_text(separator='\n').strip()
        # summary_tag = book_soup.select_one('.txtContentText')
        # summary = summary_tag.get_text(separator='\n').strip() if summary_tag is not None else ''
        lines.append({
            'title':title, 'author':author, 'company':company, 'price':price, 'imageUrl':imageUrl, 'summary':summary
        })
    except:
        continue

0  1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  21  22  23  

In [6]:
df = pd.DataFrame(lines)
len(df)

11

#### 모든 페이지에 있는 데이터

In [7]:
lines = []
for page in tqdm(range(1, 556)):               # 총 페이지가 555 페이지
    url = f'{base_url}{main_url}&pageNumber={page}'
    result = requests.get(url)
    soup = BeautifulSoup(result.text)
    lis = soup.select('#yesNewList > li')
    for li in lis:
        try:
            sub_page = li.select_one('.img_grp > a')['href']
            sub_url = f'{base_url}{sub_page}'
            res = requests.get(sub_url)
            book_soup = BeautifulSoup(res.text)
            imageUrl = book_soup.select_one('img.gImg')['src']
            title = book_soup.select_one('h2.gd_name').text.strip()
            author = book_soup.select_one('.gd_auth').text.strip()
            company = book_soup.select_one('.gd_pub').text.strip()
            price = book_soup.select_one('em.yes_m').text.strip()
            price = int(price[:-1].replace(',', ''))
            summary = book_soup.select_one('.txtContentText').get_text(separator='\n').strip()
            lines.append({
                'title':title, 'author':author, 'company':company, 'price':price, 'imageUrl':imageUrl, 'summary':summary
            })
        except:
            continue

  0%|          | 0/555 [00:00<?, ?it/s]

100%|██████████| 555/555 [1:45:54<00:00, 11.45s/it]


In [8]:
df = pd.DataFrame(lines)
len(df)

11017

In [9]:
df.drop_duplicates(inplace=True)
len(df)

11017

In [10]:
df.to_csv('../data/20241114_yes24_국내도서_새로나온_상품.csv', index=False)